# iBioFoundry - Golden Gate Assembly - OT2
author: Camillo Moschner | version: 2.0 | date: 23.01.2022 | license = 

What this code does:
- Takes a design file ('factorial' or 'defined'), creates the assmbly designs, calculates liquid handling steps, and finally exectues the steps.

# 1- Metadata

In [1]:
metadata = {
    'protocolName': '# Automated Design and/or Assembly of Golden Gate Reactions',
    'author': 'Camillo Moschner <cm967@cam.ac.uk> / <camillo.moschner@gmail.com>',
    'description': 'Programme for the automated design and/or assembly of Golden Gate reactions',
    'apiLevel': '2.10',
    'Date': '23.05.2023',
    'pipette_configuration':{'left':'p20_single_gen2',
                             'right':'p300_single_gen2'}
    }

## Assembly Master Inputs
The following cell requires changing between experimental setups (and, for simplicity for the user, is supposed to be the only part of the protocol that requires any modification).

In [2]:
# establish whether you are performing a simulation (e.g. at the design stage) or are executing the script on the OT-2
script_mode = 'simulation' # 'simulation' or 'execution'
# input data
experiment_name = 'PREEx'
design_file_directory = 'input1a_design_file_factorial_JUMP_PREEx.csv'
reagent_definitions_dir = 'input1b_reagent_definitions_MoClo_CIDAR_LVL1_BsaI.csv'

# choose naming convention for newly created plasmids:
first_part_ID = ('pPREEX', 0)
# choose plate formats:
ark_plate_format = 'biorad_96_wellplate_200ul_pcr' #'corning_384_wellplate_112ul_flat' #'384PP_AQ_BP2'
interm_stocks_plate_format = 'opentrons_24_tuberack_eppendorf_1.5ml_safelock_snapcap'
assembly_plate_format = 'opentrons_96_aluminumblock_generic_pcr_strip_200ul' # opentrons_96_aluminumblock_biorad_wellplate_200ul
assembly_plate_column_usage = 'all' # allows you to selectively only use 'odd'- or 'even'-numbered columns or 'all'; useful for 0.2 ml tubes w/ snapcaps
master_mix_position = 'D6' # in intermediate stock plate

# 2- Import Statements

In [3]:
import numpy as np
from itertools import chain, product
import pandas as pd
from IPython.display import display, clear_output, Audio, display
from copy import deepcopy
import time
from datetime import date
import math
from copy import deepcopy
import os

from iBioFoundry_helper import *
from opentrons import protocol_api, execute
from opentrons import simulate

In [4]:
# establish sole Jupyter Notebook control 
if script_mode == 'execution':
    try:
        os.system("systemctl stop opentrons-robot-server")
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    except:
        protocol = execute.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in exectution mode!\nYou should see the lights turn on and hear the gantry homing!")
elif script_mode == 'simulation':
    protocol = simulate.get_protocol_api(metadata['apiLevel']);
    print("You are operating this script in simulation mode!")
protocol.set_rail_lights(True)
protocol.home()

/Users/camillomoschner/.opentrons/robot_settings.json not found. Loading defaults
/Users/camillomoschner/.opentrons/deck_calibration.json not found. Loading defaults


You are operating this script in simulation mode!


# 3- Function Definitions

In [5]:
def smart_transfer_liquid(transfer_vol, aspiration_pos,dispensation_pos,
                          pick_up_tip=True, touch_tip_before=True,
                          cyclical_mix_before=None,mix_before_reps=2,mix_rate=0.5,
                          asp_bot_clearance=1.5,dispens_bot_clearance=1.5,
                          asp_rate=0.8,disp_rate=0.5,
                          cyclical_mix_after=None,mix_after=None,
                          dispens_top_offset=-2, blow_out=False, air_gap=False, touch_tip=True, touch_miniscus=None,
                          drop_tip=True,**pipette_info):
    """
    Liquid transfer that automatically identifies which tip to use.
    """
    if (transfer_vol >= p20_left.min_volume) & (transfer_vol <= p20_left.max_volume):
        pipette_to_use = p20_left
    elif (transfer_vol > p300_right.min_volume) & (transfer_vol <= p300_right.max_volume):
        pipette_to_use = p300_right
    else:
        print(f"WARNING: transfer volume not manageable")
    if pick_up_tip==True:
        pipette_to_use.pick_up_tip()
    if cyclical_mix_before != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_before[0]):
            pipette_to_use.aspirate(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[2]))
            pipette_to_use.dispense(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[3]))
    if mix_before_reps != 0:
        pipette_to_use.mix(mix_before_reps,transfer_vol*0.8,aspiration_pos.bottom(asp_bot_clearance),mix_rate)
    # aspiration
    pipette_to_use.aspirate(transfer_vol,aspiration_pos.bottom(asp_bot_clearance), asp_rate)
    #protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
    pipette_to_use.move_to(aspiration_pos.top())
    #protocol.max_speeds['Z'] = None # reset z axis speed
    if air_gap != False:
        pipette_to_use.air_gap(air_gap)
    protocol.delay(seconds=1)
    if touch_tip_before == True:
        pipette_to_use.touch_tip(v_offset=-2, speed=150)
    # dispensation
    pipette_to_use.dispense(transfer_vol,dispensation_pos.bottom(dispens_bot_clearance),disp_rate)
    if cyclical_mix_after != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        for i in np.arange(cyclical_mix_after[0]):
            pipette_to_use.aspirate(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[2]))
            pipette_to_use.dispense(cyclical_mix_after[1],dispensation_pos.bottom(cyclical_mix_after[3]))
    if mix_after != None:
        pipette_to_use.mix(mix_after[0],mix_after[1], dispensation_pos.bottom(mix_after[2]),mix_rate)
    pipette_to_use.move_to(dispensation_pos.top())
    protocol.delay(seconds=2)
    if blow_out == True:
        pipette_to_use.blow_out(dispensation_pos.top(dispens_top_offset))
    if touch_tip == True:
        pipette_to_use.touch_tip(v_offset=dispens_top_offset, speed=150)
    if touch_miniscus != None:
        protocol.max_speeds['Z'] = 10  # limit z axis to 50 mm/s to avoid solution adhereing to outer tip surface
        pipette_to_use.move_to(dispensation_pos.bottom(touch_miniscus))
        protocol.max_speeds['Z'] = None # reset z axis speed
    if drop_tip==True:
        pipette_to_use.drop_tip()

def smart_distribute_liquid(single_distr_vol, aspiration_pos,dispensation_pos_list,
                            asp_bot_clearance=1.1,dispens_bot_clearance=1.8,
                            asp_flow_rate = 5,disp_flow_rate = 5,
                            cyclical_mix_before=None, touch_tip_bool=True):
    """
    Liquid distribution from one aspiration well into multiple destination wells that automatically identifies which pipette to use.
    """
    if (single_distr_vol >= 1) & (single_distr_vol <= 20):
        pipette_to_use = p20_left
    elif (single_distr_vol > 20):
        pipette_to_use = p300_right
    else:
        print(f"WARNING: transfer volume not manageable")
    pipette_to_use.flow_rate.aspirate = asp_flow_rate
    pipette_to_use.flow_rate.dispense = disp_flow_rate
    if cyclical_mix_before != None: # give tuple (repetitions, mix_volume, well_bottom_clearance)
        pipette_to_use.pick_up_tip()
        for i in np.arange(cyclical_mix_before[0]):
            pipette_to_use.aspirate(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[2]))
            pipette_to_use.dispense(cyclical_mix_before[1],aspiration_pos.bottom(cyclical_mix_before[3]))
        pipette_to_use.drop_tip()
    pipette_to_use.well_bottom_clearance.aspirate = asp_bot_clearance
    pipette_to_use.well_bottom_clearance.dispense = dispens_bot_clearance
    pipette_to_use.distribute(single_distr_vol, aspiration_pos,dispensation_pos_list, touch_tip=touch_tip_bool)
    pipette_to_use.well_bottom_clearance.dispense = 1
    pipette_to_use.well_bottom_clearance.aspirate = 1
    pipette_to_use.flow_rate.aspirate = 7.56
    pipette_to_use.flow_rate.dispense = 7.56

---
# 4- Deck assignments

In [8]:
"""Deck creation"""
deck_slot_list =list(np.arange(1,12))
deck_slot_list.append('bin')
deck_slot_df = pd.DataFrame( np.flip(np.array(deck_slot_list).reshape(4,3), axis=0) )

## i. Labware
Resource: [Opentrons Labware Library Webpage](https://labware.opentrons.com/?_gl=1*1iyc1t4*_ga*MTU1MTM1NzU5MS4xNjE2OTczMzY5*_ga_GNSMNLW4RY*MTYzMDYxNzY1NS4yMC4wLjE2MzA2MTc2NTUuMA..&_ga=2.110416899.767557364.1630617656-1551357591.1616973369)
### a. Modules

In [9]:
temp_mod_name  = 'temperature module gen2'
temp_mod = protocol.load_module(temp_mod_name,'3')
deck_slot_df.iloc[3,2] = temp_mod_name

### b. Racks & Plates

In [10]:
tiprack20a = protocol.load_labware('opentrons_96_tiprack_20ul','8')
tiprack20b = protocol.load_labware('opentrons_96_tiprack_20ul','11')
tiprack20c = protocol.load_labware('opentrons_96_tiprack_20ul','7')
tiprack300a = protocol.load_labware('opentrons_96_tiprack_300ul','10')
# tuberacks & plates
moladj_parts_plate = protocol.load_labware(ark_plate_format,'5') # assigned in cell no. 5
interm_stocks_rack = protocol.load_labware(interm_stocks_plate_format,'1') #  biorad_96_wellplate_200ul_pcr
assembly_plate = temp_mod.load_labware(assembly_plate_format,'2') # opentrons_96_aluminumblock_biorad_wellplate_200ul
water_rack = protocol.load_labware('opentrons_6_tuberack_falcon_50ml_conical','4') 

In [11]:
labware_list = [tiprack20a,tiprack20b,tiprack20c, tiprack300a, 
                moladj_parts_plate, interm_stocks_rack, assembly_plate, water_rack]
_ =[update_labware(deck_slot_df, labware_item) for labware_item in labware_list]
# visually inspect deck
print("\nFinal OT-2 Deck Configuration:");(deck_slot_df)


Final OT-2 Deck Configuration:


,0,1,2
0,10-opentrons_96_tiprack_300ul,11-opentrons_96_tiprack_20ul,bin
1,7-opentrons_96_tiprack_20ul,8-opentrons_96_tiprack_20ul,9
2,4-opentrons_6_tuberack_falcon_50ml_conical,5-biorad_96_wellplate_200ul_pcr,6
3,1-opentrons_24_tuberack_eppendorf_1.5ml_safelo...,2,temperature module gen2


## ii. Pipettes

Resource: [Opentrons Pipettes API Webpage](https://docs.opentrons.com/v2/new_pipette.html)

In [12]:
p20_left = protocol.load_instrument('p20_single_gen2','left', tip_racks=[tiprack20a,tiprack20b,tiprack20c])
p300_right = protocol.load_instrument('p300_single_gen2','right', tip_racks=[tiprack300a])

## iii. Molarity-adjusted parts info integration

In [13]:
sample_definitionslayout_dir = f'0_ark_plate_info_{experiment_name}{os.path.sep}1_ark_plate_info.csv'
sample_definitions_df = pd.read_csv(sample_definitionslayout_dir)
sample_definitions_df['position_tuple'] = sample_definitions_df['moladj_parts_positions'].apply(lambda name: (name[0],int(name[1:])))

# create new moladj_parts_plate_obj and 
moladj_parts_plate_obj = Plate(identify_plate(ark_plate_format, 'plate_format'))
assembly_plate_obj = Plate(identify_plate(assembly_plate_format, 'plate_format'),only_columns=assembly_plate_column_usage)

# update the new moladj_parts_plate_obj with the precise location of the prepped, molarity-adjusted parts
for part_info in [sample_definitions_df.iloc[row,:] for row in range(len(sample_definitions_df))]:
    moladj_parts_plate_obj.layout.loc[part_info['position_tuple']]=part_info['name']
sample_definitions_df

,name,concentration (nM),total volume (ul),moladj_parts_positions,ark_plate_format,prep_date,position_tuple
0,P_T7(BBF10K_003378)_15nM,15.0,110.02,A1,96,23/05/2023,"(A, 1)"
1,P_T7_lacO(BBF10K_003379)_15nM,15.0,106.66,B1,96,23/05/2023,"(B, 1)"
2,RBS_BT1_BCD2(BBF10K_003384)_15nM,15.0,122.92,C1,96,23/05/2023,"(C, 1)"
3,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,15.0,169.70,D1,96,23/05/2023,"(D, 1)"
4,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,15.0,98.70,E1,96,23/05/2023,"(E, 1)"
5,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,15.0,129.82,F1,96,23/05/2023,"(F, 1)"
6,Ntag1_R5(BBF10K_003399)_30nM,30.0,67.92,G1,96,23/05/2023,"(G, 1)"
7,Ntag2_TEVcut(BBF10K_003418)_30nM,30.0,67.62,H1,96,23/05/2023,"(H, 1)"
8,Ntag_His_TEVcut(BBF10K_003407_15nM,15.0,120.96,A2,96,23/05/2023,"(A, 2)"
9,CDS_Eco31I(BBF10K_003292)_15nM,15.0,131.38,B2,96,23/05/2023,"(B, 2)"


---
# 5- LH Step Calculations 
## Design file reading

In [14]:
design_info_int = pd.read_csv(design_file_directory,nrows=0).columns.tolist()
design_info = list(chain.from_iterable([x.split(':') for x in design_info_int]))
design_info = {design_info[i]: design_info[i+1] for i in range(0, len(design_info), 2)}
if (design_info['design type'] == 'factorial') or (design_info['design type'] == 'defined'):
    _=[print(f"{key} : {design_info[key]}") for key in design_info.keys()]
else:
    print(f"WARNING: Design type not specified!")

design type : factorial
GG standard : uLoop
assembly level : LVL1
LHS : Opentrons_OT2
date : 10.05.2023
Unnamed :  5


In [15]:
design_file_df = pd.read_csv(design_file_directory,header=1) 
# create complete list of DNA parts required:
complete_DNA_parts_list_required = list(chain.from_iterable([ sorted(set(notna_data(design_file_df[name]))) for name in design_file_df.columns ]))
display(design_file_df)
#display(complete_DNA_parts_list_required)
print(f"\n -> {len(complete_DNA_parts_list_required)} parts needed \n")

,Module A | Part 1: Promoter,Module A | Part 2: RBS,Module A | Part 3 & 4: Tag1and2,Module A | Part 5: CDS,Module A | Part 6: Terminator,Module A | Part 7: LVL1 Destination Vector
0,P_T7(BBF10K_003378)_15nM,RBS_BT1_BCD2(BBF10K_003384)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,CDS_Eco31I(BBF10K_003292)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM
1,P_T7_lacO(BBF10K_003379)_15nM,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,CDS_EcoRI(BBF10K_003281)_15nM,NaN,NaN
2,NaN,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,NaN,CDS_NotI(BBF10K_003300)_15nM,NaN,NaN
3,NaN,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,NaN,CDS_PstI(BBF10K_003283)_15nM,NaN,NaN
4,NaN,NaN,NaN,CDS_SapI(BBF10K_003295)_15nM,NaN,NaN
5,NaN,NaN,NaN,CDS_XbaI(BBF10K_003298)_15nM,NaN,NaN



 -> 16 parts needed 



## Create combinations

In [16]:
combinations_list = list(product(*[notna_data(design_file_df[part]) for part in design_file_df.columns])) 

if design_info['design type'] == 'factorial':
    combinations_df = pd.DataFrame(combinations_list,columns=design_file_df.columns.tolist())
elif design_info['design type'] == 'defined':
    combinations_df = deepcopy(design_file_df)
combinations_no = len(combinations_df)
part_per_assembly_no = len(design_file_df.columns)
# identify most diverse part
comb_description = combinations_df.describe()
part_uniques = pd.DataFrame(comb_description.loc['unique',:])
most_diverse_part = part_uniques.loc[part_uniques['unique'] == part_uniques['unique'].max()]
if int(part_uniques.var()) == 0:
    print(f"All parts are equi-present & therefore there is no \'most_diverse_part\'. Since one has to be defined for further steps though the first part will have been assigned this variable \n")
else:
    print(f"The current design would create - {combinations_no} - different assembly combinations/new plasmids.\n\'{most_diverse_part.index[0]}\' is the most diverse part.\n")
comb_description

The current design would create - 96 - different assembly combinations/new plasmids.
'Module A | Part 5: CDS' is the most diverse part.



,Module A | Part 1: Promoter,Module A | Part 2: RBS,Module A | Part 3 & 4: Tag1and2,Module A | Part 5: CDS,Module A | Part 6: Terminator,Module A | Part 7: LVL1 Destination Vector
count,96,96,96,96,96,96
unique,2,4,2,6,1,1
top,P_T7(BBF10K_003378)_15nM,RBS_BT1_BCD2(BBF10K_003384)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,CDS_Eco31I(BBF10K_003292)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM
freq,48,24,48,16,96,96


---

## Assembly plate check
Check that there is enough space in the assembly plate for all the calculated assemblies.

In [17]:
usable_destination_plate_pos_list = [index+str(column) for column in assembly_plate_obj.layout.columns for index in assembly_plate_obj.layout.index ]
no_usable_destination_plate_positions = len(assembly_plate_obj.layout.columns)*len(assembly_plate_obj.layout.index)
if combinations_no <= no_usable_destination_plate_positions:
    print(f"✅ All clear: You are trying to assemble {combinations_no} constructs and have {len(usable_destination_plate_pos_list)} positions available on the assembly plate.")
elif (combinations_no > no_usable_destination_plate_positions) and (combinations_no <= 96):
    print(f"""\nWARNING: You are trying to assemble {combinations_no} constructs but only have {len(usable_destination_plate_pos_list)} positions available on the assembly plate!
    -> You have to use the full assembly_plate! e.g. with a BioRad 96-well PCR plate.\n""")
elif combinations_no > 96:
    print(f"""\nWARNING: You are trying to assemble {combinations_no} constructs but only have {len(usable_destination_plate_pos_list)} positions available on the assembly plate!
    -> You have to use the full assembly_plate in multiple iterations! e.g. use 2x BioRad 96-well PCR plates.\n""")

✅ All clear: You are trying to assemble 96 constructs and have 96 positions available on the assembly plate.


## Create intermediate stock IDs

In [18]:
# create intermediate stock IDs
if design_info['design type'] == 'factorial':
    try:
        combinations_df['inter_stock_ID'] = combinations_df['inter_stock_ID'] # in case you run this cell multiple times (otherwise would expand combinations_df['inter_stock_ID'] on every call)
    except:
        inter_stock_ID = ['|'.join(combinations_df.loc[:,combinations_df.columns!=most_diverse_part.index[0]].loc[row_idx,:].tolist()) for row_idx in np.arange(combinations_no)]
    combinations_df['inter_stock_ID'] = inter_stock_ID
    interm_stocks_list = list(combinations_df['inter_stock_ID'].unique())
    max_interm_stocks = len(interm_stocks_list)
    interm_stocks_df = part_uniques.loc[part_uniques['unique'].index!=most_diverse_part['unique'].index[0]]
    display(interm_stocks_df)
    print(f"=> {max_interm_stocks} intermediate_stocks are needed for these {combinations_no} assemblies!\n")
# assign each assembly an final assembly_tube, assembly_ID and assembly_description
combinations_df['assembly_ID'] = create_part_IDs_list(first_part_ID,combinations_no)
extend_fill_plate_df_with_list(assembly_plate_obj.layout, combinations_df.assembly_ID.to_list(), fill_start_position='A1', inplace=True, fill_first='columns') # change to 'columns' to assemble left-right rather than top-down
combinations_df['assembly_tube'] = list(assembly_plate_obj.occupied_wells.keys())#usable_destination_plate_pos_list[:combinations_no]

try:
    combinations_df['assembly_description'] = combinations_df['assembly_description'] # in case you run this cell multiple times (otherwise would expand combinations_df['assembly_description'] on every call)
except:
    assembly_descriptions = ['|'.join(combinations_df.loc[:,design_file_df.columns].loc[row_idx,:].tolist()) for row_idx in np.arange(combinations_no)]
combinations_df['assembly_description'] = assembly_descriptions
combinations_df['assembly_date'] = [str(date.today()) for i in np.arange(combinations_no)]
# move important info to the fron of the spreadsheet
for column_name in ['assembly_description', 'assembly_ID']:
    first_column = combinations_df.pop(column_name)
    combinations_df.insert(0, column_name, first_column)

,unique
Module A | Part 1: Promoter,2
Module A | Part 2: RBS,4
Module A | Part 3 & 4: Tag1and2,2
Module A | Part 6: Terminator,1
Module A | Part 7: LVL1 Destination Vector,1


=> 16 intermediate_stocks are needed for these 96 assemblies!



In [19]:
combinations_df

,assembly_ID,assembly_description,Module A | Part 1: Promoter,Module A | Part 2: RBS,Module A | Part 3 & 4: Tag1and2,Module A | Part 5: CDS,Module A | Part 6: Terminator,Module A | Part 7: LVL1 Destination Vector,inter_stock_ID,assembly_tube,assembly_date
0,pPREEX0,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,P_T7(BBF10K_003378)_15nM,RBS_BT1_BCD2(BBF10K_003384)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,CDS_Eco31I(BBF10K_003292)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,A1,2023-06-04
1,pPREEX1,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,P_T7(BBF10K_003378)_15nM,RBS_BT1_BCD2(BBF10K_003384)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,CDS_EcoRI(BBF10K_003281)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,B1,2023-06-04
2,pPREEX2,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,P_T7(BBF10K_003378)_15nM,RBS_BT1_BCD2(BBF10K_003384)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,CDS_NotI(BBF10K_003300)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,C1,2023-06-04
3,pPREEX3,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,P_T7(BBF10K_003378)_15nM,RBS_BT1_BCD2(BBF10K_003384)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,CDS_PstI(BBF10K_003283)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,D1,2023-06-04
4,pPREEX4,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,P_T7(BBF10K_003378)_15nM,RBS_BT1_BCD2(BBF10K_003384)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,CDS_SapI(BBF10K_003295)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,E1,2023-06-04
...,...,...,...,...,...,...,...,...,...,...,...
91,pPREEX91,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_pel...,P_T7_lacO(BBF10K_003379)_15nM,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,CDS_EcoRI(BBF10K_003281)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_pel...,D12,2023-06-04
92,pPREEX92,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_pel...,P_T7_lacO(BBF10K_003379)_15nM,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,CDS_NotI(BBF10K_003300)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_pel...,E12,2023-06-04
93,pPREEX93,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_pel...,P_T7_lacO(BBF10K_003379)_15nM,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,CDS_PstI(BBF10K_003283)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_pel...,F12,2023-06-04
94,pPREEX94,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_pel...,P_T7_lacO(BBF10K_003379)_15nM,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,CDS_SapI(BBF10K_003295)_15nM,T_TZ(BBF10K_003477)_15nM,DV_pTi_5nM,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_pel...,G12,2023-06-04


## Interm. stock assignment

In [20]:
# update part_rack with where to put the intermediate_stock_tubes and the GGA master mix
interm_stocks_rack_obj = Plate(identify_plate(interm_stocks_plate_format, 'plate_format'))
if design_info['design type'] == 'factorial':
    extend_fill_plate_df_with_list(interm_stocks_rack_obj.layout, interm_stocks_list, 
                                   fill_start_position='A1', inplace=True, fill_first='columns')
interm_stocks_rack_obj.layout.loc[convert_to_position_tuple(master_mix_position)] = 'GGA_MM'

## - Visual Deck Inspection

In [21]:
print("__Part Plate__")
display(moladj_parts_plate_obj.layout)
print("\n__Intermediate Stock Plate__")
display(interm_stocks_rack_obj.layout)
print("\n__Assembly Plate__")
display(assembly_plate_obj.layout)

__Part Plate__


,1,2,3,4,5,6,7,8,9,10,11,12
A,P_T7(BBF10K_003378)_15nM,Ntag_His_TEVcut(BBF10K_003407_15nM,DV_pTi_7.5nM,0,0,0,0,0,0,0,0,0
B,P_T7_lacO(BBF10K_003379)_15nM,CDS_Eco31I(BBF10K_003292)_15nM,DV_pTi_5nM,0,0,0,0,0,0,0,0,0
C,RBS_BT1_BCD2(BBF10K_003384)_15nM,CDS_EcoRI(BBF10K_003281)_15nM,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,0,0,0,0,0,0,0,0,0
D,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,CDS_NotI(BBF10K_003300)_15nM,0,0,0,0,0,0,0,0,0,0
E,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,CDS_PstI(BBF10K_003283)_15nM,0,0,0,0,0,0,0,0,0,0
F,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,CDS_SapI(BBF10K_003295)_15nM,0,0,0,0,0,0,0,0,0,0
G,Ntag1_R5(BBF10K_003399)_30nM,CDS_XbaI(BBF10K_003298)_15nM,0,0,0,0,0,0,0,0,0,0
H,Ntag2_TEVcut(BBF10K_003418)_30nM,T_TZ(BBF10K_003477)_15nM,0,0,0,0,0,0,0,0,0,0



__Intermediate Stock Plate__


,1,2,3,4,5,6
A,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2_OmpT(BBF...,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2(BBF...,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_Omp...,0,0
B,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_0...,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2_OmpT(BBF...,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2(BBF...,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_Omp...,0,0
C,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2_DsbA(BBF...,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2_pelB(BBF...,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_Dsb...,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_pel...,0,0
D,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2_DsbA(BBF...,P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2_pelB(BBF...,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_Dsb...,P_T7_lacO(BBF10K_003379)_15nM|RBS_BT1_BCD2_pel...,0,GGA_MM



__Assembly Plate__


,1,2,3,4,5,6,7,8,9,10,11,12
A,pPREEX0,pPREEX8,pPREEX16,pPREEX24,pPREEX32,pPREEX40,pPREEX48,pPREEX56,pPREEX64,pPREEX72,pPREEX80,pPREEX88
B,pPREEX1,pPREEX9,pPREEX17,pPREEX25,pPREEX33,pPREEX41,pPREEX49,pPREEX57,pPREEX65,pPREEX73,pPREEX81,pPREEX89
C,pPREEX2,pPREEX10,pPREEX18,pPREEX26,pPREEX34,pPREEX42,pPREEX50,pPREEX58,pPREEX66,pPREEX74,pPREEX82,pPREEX90
D,pPREEX3,pPREEX11,pPREEX19,pPREEX27,pPREEX35,pPREEX43,pPREEX51,pPREEX59,pPREEX67,pPREEX75,pPREEX83,pPREEX91
E,pPREEX4,pPREEX12,pPREEX20,pPREEX28,pPREEX36,pPREEX44,pPREEX52,pPREEX60,pPREEX68,pPREEX76,pPREEX84,pPREEX92
F,pPREEX5,pPREEX13,pPREEX21,pPREEX29,pPREEX37,pPREEX45,pPREEX53,pPREEX61,pPREEX69,pPREEX77,pPREEX85,pPREEX93
G,pPREEX6,pPREEX14,pPREEX22,pPREEX30,pPREEX38,pPREEX46,pPREEX54,pPREEX62,pPREEX70,pPREEX78,pPREEX86,pPREEX94
H,pPREEX7,pPREEX15,pPREEX23,pPREEX31,pPREEX39,pPREEX47,pPREEX55,pPREEX63,pPREEX71,pPREEX79,pPREEX87,pPREEX95


## Part usage summary

In [22]:
DNA_part_counts = []
_=[DNA_part_counts.append(combinations_df[part_name].value_counts()) for part_name in design_file_df.columns.tolist()]
part_names, part_counts = [], []
[part_names.append(DNA_part_counts[idx].index.tolist()) for idx in np.arange(len(DNA_part_counts))], [part_counts.append(DNA_part_counts[idx].tolist()) for idx in np.arange(len(DNA_part_counts))]
part_names, part_counts = list(chain.from_iterable(part_names)), list(chain.from_iterable(part_counts))
part_types = [find_df_coordinates(design_file_df,part_name,tuple_result=True)[-1][-1] if type(find_df_coordinates(design_file_df,part_name,tuple_result=True))!=tuple else find_df_coordinates(design_file_df,part_name,tuple_result=True)[-1] for part_name in part_names]

# convert part usage statistics into pandas.DataFrame, calculate insurance volumes for each and show each's position in the parts_rack
part_usage_summary_df = pd.DataFrame([part_types,part_names,part_counts]).T
part_usage_summary_df.rename(columns={0: 'part_type', 1: 'part_name', 2: 'usage (µl)'},inplace=True)
part_usage_summary_df['insurance_vol'] = part_usage_summary_df['usage (µl)']*1.1+20 # 20 µl dead volume (to insure against calibration issues & well_bottom_clearance problems)
part_usage_summary_df['position'] = [find_df_coordinates(moladj_parts_plate_obj.layout,part_name,tuple_result=True)[0]+str(find_df_coordinates(moladj_parts_plate_obj.layout,part_name,tuple_result=True)[1]).zfill(2) for part_name in part_usage_summary_df['part_name']]
part_usage_summary_df.sort_values(by=['position'],inplace=True)
part_usage_summary_df.reset_index(drop=True,inplace=True)
# check availability of the volume required for all parts in the molarity-adjusted parts plate
availability_checkup = []
leftover_vol = []
for required_part_info in [part_usage_summary_df.iloc[row_no,:] for row_no in range(len(part_usage_summary_df))]:
    current_part_name = required_part_info['part_name']
    current_insurance_vol = required_part_info['insurance_vol']
    current_part_in_plate_info = sample_definitions_df.loc[sample_definitions_df.name == current_part_name ]
    leftover_in_plate_after_assembly = current_part_in_plate_info['total volume (ul)'].iloc[0] - current_insurance_vol
    if leftover_in_plate_after_assembly > 0:
        availability_checkup.append('ok')
    else: 
        print(f"WARNING: {current_part_name} has not enough miniprep available!")
        availability_checkup.append('WARNING')
    leftover_vol.append(round(leftover_in_plate_after_assembly,2))
part_usage_summary_df['availability'] = availability_checkup
part_usage_summary_df['leftover'] = leftover_vol
part_usage_summary_df['leftover_check'] = ['ok' if vol > 15 else 'WARNING' for vol in leftover_vol] # defines the dead vol -> requires checking
part_usage_summary_df

,part_type,part_name,usage (µl),insurance_vol,position,availability,leftover,leftover_check
0,Module A | Part 1: Promoter,P_T7(BBF10K_003378)_15nM,48,72.8,A01,ok,37.22,ok
1,Module A | Part 3 & 4: Tag1and2,Ntag_His_TEVcut(BBF10K_003407_15nM,48,72.8,A02,ok,48.16,ok
2,Module A | Part 1: Promoter,P_T7_lacO(BBF10K_003379)_15nM,48,72.8,B01,ok,33.86,ok
3,Module A | Part 5: CDS,CDS_Eco31I(BBF10K_003292)_15nM,16,37.6,B02,ok,93.78,ok
4,Module A | Part 7: LVL1 Destination Vector,DV_pTi_5nM,96,125.6,B03,ok,54.40,ok
5,Module A | Part 2: RBS,RBS_BT1_BCD2(BBF10K_003384)_15nM,24,46.4,C01,ok,76.52,ok
6,Module A | Part 5: CDS,CDS_EcoRI(BBF10K_003281)_15nM,16,37.6,C02,ok,106.16,ok
7,Module A | Part 3 & 4: Tag1and2,Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF1...,48,72.8,C03,ok,47.20,ok
8,Module A | Part 2: RBS,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,24,46.4,D01,ok,123.30,ok
9,Module A | Part 5: CDS,CDS_NotI(BBF10K_003300)_15nM,16,37.6,D02,ok,77.94,ok


## Reagent preparation instructions

In [23]:
reagent_definitions_df = pd.read_csv(reagent_definitions_dir, index_col=0)
reagent_preptube = interm_stocks_rack[master_mix_position]

In [24]:
dead_vol = 15 # units: µl
if combinations_no < 40:
    assembly_surplus = 8 # %
else:
    assembly_surplus = 5 # %
# calculate combined reagent vol
reagent_vol = reagent_definitions_df.loc[list(reagent_definitions_df.index[reagent_definitions_df.index!='water1']),:]['1x_work_vol_10ul'].sum()
# calculate water vol needed & update reagent definitions
one_x_water = (10-reagent_vol)-part_per_assembly_no
combined_reagent_vol = reagent_vol+one_x_water
reagent_definitions_df.loc['water1','1x_work_vol_10ul']=one_x_water
reagent_definitions_df
# scenario: a user has added more than 8 parts into the assembly -> even though the code could theoretically cope with it, 
# the OT-2 cannot pipette volumes smaller than 1 µl and therefore the number of parts is limited by reaction_volume (i.e. 10)-sum(reagent_volumes)
if one_x_water < 0.25:
    raise SystemExit(f"WARNING: Water volume is {one_x_water} - that is too low to be pipetted!")
reagent_definitions_df['1x_prep_vol'] = round(reagent_definitions_df['1x_work_vol_10ul']*(assembly_surplus/100+1),2) # 40% extra liquid to mitigate any pipetting inaccuracies and liquid loss due to pipette retention
# integrate surplus and dead volume
prep_col_name, dead_vol_comp_factor = str(combinations_no)+'x_prep_vol', dead_vol/reagent_definitions_df['1x_work_vol_10ul'].sum()
reagent_definitions_df[prep_col_name] = round(reagent_definitions_df['1x_prep_vol']*combinations_no,2)
reagent_definitions_df[prep_col_name] = reagent_definitions_df[prep_col_name]+reagent_definitions_df['1x_work_vol_10ul']*dead_vol_comp_factor
print(f"\nA master mix of {round(reagent_definitions_df[prep_col_name].sum(),2)} µl in volume has to be prepared and put into position {reagent_preptube}.\n");display(reagent_definitions_df);


A master mix of 417.24 µl in volume has to be prepared and put into position D6 of Opentrons 24 Tube Rack with Eppendorf 1.5 mL Safe-Lock Snapcap on 1.



,1x_work_vol_10ul,1x_prep_vol,96x_prep_vol
water1,1.5,1.58,157.305
T4_DNAligase_buffer,1.0,1.05,104.550
BSA,0.5,0.52,51.795
T4_DNAligase,0.5,0.52,51.795
BsaI-HFv2,0.5,0.52,51.795


## Summary Spreadsheets

In [25]:
save_spreadsheet_answer = input("Do you want to save the summary spreadsheets? (yes / no)\n answer:")
if save_spreadsheet_answer == 'yes':
    try:
        os.makedirs('1_GGA')
    except:
        pass
    # save prepped equimolar sample information
    combinations_df.to_csv('1_GGA/assemblies_summary.csv',index=False)
    reagent_definitions_df.to_csv('1_GGA/reagent_tracker_summary.csv',index=False)
    print(f"The GGA summary spreadsheets have been saved to \'1_GGA{os.path.sep}\'!")
else:
    print(f"\nYou have not saved the GGA summary spreadsheets!\n")

Do you want to save the summary spreadsheets? (yes / no)
 answer: yes


The GGA summary spreadsheets have been saved to '1_GGA/'!


In [26]:
temp_mod.set_temperature(6)
# allDone()

---
# 6- Execution

## i. Reagent transfer

In [28]:
print(f"Assembly master mix transfer...\n  {combined_reagent_vol} µl of MM from {reagent_preptube}\n   -> ")
print(f"   {assembly_plate}\n  {[assembly_plate[combinations_df.loc[assembly_idx,'assembly_tube']].well_name for assembly_idx in range(combinations_no)]}")
if design_info['design type'] == 'factorial':
    smart_distribute_liquid(combined_reagent_vol,
                            reagent_preptube,
                            [assembly_plate[combinations_df.loc[assembly_idx,'assembly_tube']] for assembly_idx in range(combinations_no)],
                            dispens_bot_clearance=1.2,touch_tip_bool=False,
                            asp_flow_rate = 3, disp_flow_rate = 3)

Assembly master mix transfer...
  4.0 µl of MM from D6 of Opentrons 24 Tube Rack with Eppendorf 1.5 mL Safe-Lock Snapcap on 1
   -> 
   2 on Temperature Module GEN2 on 3
  ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'A9', 'B9', 'C9', 'D9', 'E9', 'F9', 'G9', 'H9', 'A10', 'B10', 'C10', 'D10', 'E10', 'F10', 'G10', 'H10', 'A11', 'B11', 'C11', 'D11', 'E11', 'F11', 'G11', 'H11', 'A12', 'B12', 'C12', 'D12', 'E12', 'F12', 'G12', 'H12']


## ii. Stock & part distributions (F)

In [29]:
# integrate volume tracking
sample_definitions_df[f'sample_vol_used_{str(date.today())}'] = [float(0) for i in range(len(sample_definitions_df))]

In [30]:
"""
Create intermediate stocks
"""
if design_info['design type'] == 'factorial':
    for intermediate_stock in interm_stocks_list:
        current_interm_stock_data_df = combinations_df.loc[combinations_df['inter_stock_ID']==intermediate_stock]
        transfer_vol = len(current_interm_stock_data_df) * 1.1 + round(dead_vol/len(combinations_df.columns),2)
        interm_stock_part_rack_well = find_df_coordinates(interm_stocks_rack_obj.layout,intermediate_stock)
        print(f"( {interm_stocks_list.index(intermediate_stock)+1} / {len(interm_stocks_list)} ) - {round(transfer_vol*len(intermediate_stock.split('|')),2)} µl of intermediate stock \'{intermediate_stock}\' assembled in _parts_rack {interm_stock_part_rack_well}_")
        for interm_stock_part in intermediate_stock.split('|'):
            part_well_location = moladj_parts_plate[find_df_coordinates(moladj_parts_plate_obj.layout,interm_stock_part)]
            part_index = sample_definitions_df.index[sample_definitions_df['name'] == interm_stock_part]
            if len(part_index) > 1:
                print("WARNING: You have multiple parts with the same name!")
                break
            else:
                part_index = part_index[0]
            print(f"       -> {round(transfer_vol,2)} µl  part \'{part_well_location.well_name}\': \'{interm_stock_part}\' -> intermedidate_stock \'{interm_stocks_rack[interm_stock_part_rack_well].well_name}\'")
            smart_transfer_liquid(transfer_vol,
                                  part_well_location,
                                  interm_stocks_rack[interm_stock_part_rack_well], 
                                  asp_rate=0.6, disp_rate=0.4,
                                  touch_tip_before=False,touch_tip=False)
            sample_definitions_df.at[part_index,f'sample_vol_used_{str(date.today())}'] -= round(transfer_vol,2)
print("\n")

( 1 / 16 ) - 39.8 µl of intermediate stock 'P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_003384)_15nM|Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF10K_003418)_15nM|T_TZ(BBF10K_003477)_15nM|DV_pTi_5nM' assembled in _parts_rack A1_
       -> 7.96 µl  part 'A1': 'P_T7(BBF10K_003378)_15nM' -> intermedidate_stock 'A1'
       -> 7.96 µl  part 'C1': 'RBS_BT1_BCD2(BBF10K_003384)_15nM' -> intermedidate_stock 'A1'
       -> 7.96 µl  part 'C3': 'Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF10K_003418)_15nM' -> intermedidate_stock 'A1'
       -> 7.96 µl  part 'H2': 'T_TZ(BBF10K_003477)_15nM' -> intermedidate_stock 'A1'
       -> 7.96 µl  part 'B3': 'DV_pTi_5nM' -> intermedidate_stock 'A1'
( 2 / 16 ) - 39.8 µl of intermediate stock 'P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_003384)_15nM|Ntag_His_TEVcut(BBF10K_003407_15nM|T_TZ(BBF10K_003477)_15nM|DV_pTi_5nM' assembled in _parts_rack B1_
       -> 7.96 µl  part 'A1': 'P_T7(BBF10K_003378)_15nM' -> intermedidate_stock 'B1'
       -> 7.96 µl  part 'C

In [31]:
"""
Distribute intermediate stocks into assembly tubes
"""
if design_info['design type'] == 'factorial':
    print(f"Assembly plate wells receiving DNA parts from intermediate stocks:")
    for intermediate_stock in interm_stocks_list:
        current_interm_stock_data_df = combinations_df.loc[combinations_df['inter_stock_ID']==intermediate_stock]
        transfer_vol = len(intermediate_stock.split('|'))
        intermediate_stock_distr_well_list = current_interm_stock_data_df['assembly_tube'].to_list()
        print(f"( {interm_stocks_list.index(intermediate_stock)+1} / {len(interm_stocks_list)} ) - {transfer_vol} µl from intermediate stock \'{intermediate_stock}\' distributed into...\n                      {intermediate_stock_distr_well_list}")
        intermediate_stock_distr_location_list = [assembly_plate[assembly_position].bottom(2) for assembly_position in intermediate_stock_distr_well_list]
        if intermediate_stock == interm_stocks_list[0]:
            smart_distribute_liquid(transfer_vol,
                                    interm_stocks_rack[find_df_coordinates(interm_stocks_rack_obj.layout,intermediate_stock)],
                                    intermediate_stock_distr_location_list,
                                    cyclical_mix_before = (5, 19.5, 1,3),
                                    dispens_bot_clearance=1.3,touch_tip_bool=False)
        else:
            smart_distribute_liquid(transfer_vol,
                                    interm_stocks_rack[find_df_coordinates(interm_stocks_rack_obj.layout,intermediate_stock)],
                                    intermediate_stock_distr_location_list,
                                    dispens_bot_clearance=1.3,touch_tip_bool=False)
print("\n")

Assembly plate wells receiving DNA parts from intermediate stocks:
( 1 / 16 ) - 5 µl from intermediate stock 'P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_003384)_15nM|Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF10K_003418)_15nM|T_TZ(BBF10K_003477)_15nM|DV_pTi_5nM' distributed into...
                      ['A1', 'B1', 'C1', 'D1', 'E1', 'F1']
( 2 / 16 ) - 5 µl from intermediate stock 'P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2(BBF10K_003384)_15nM|Ntag_His_TEVcut(BBF10K_003407_15nM|T_TZ(BBF10K_003477)_15nM|DV_pTi_5nM' distributed into...
                      ['G1', 'H1', 'A2', 'B2', 'C2', 'D2']
( 3 / 16 ) - 5 µl from intermediate stock 'P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM|Ntag1_R5(BBF10K_003399)_15nM-Ntag2_TEVcut(BBF10K_003418)_15nM|T_TZ(BBF10K_003477)_15nM|DV_pTi_5nM' distributed into...
                      ['E2', 'F2', 'G2', 'H2', 'A3', 'B3']
( 4 / 16 ) - 5 µl from intermediate stock 'P_T7(BBF10K_003378)_15nM|RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM|Ntag_His_TEV

In [32]:
"""
Dispense 1 µl of remaining, most diverse parts into assembly tubes
"""
if design_info['design type'] == 'factorial':
    print(f"Transfer of remaining, most diverse part...")
    for idx, part_name in enumerate(combinations_df[most_diverse_part.index[0]]):
        part_well_location = moladj_parts_plate[find_df_coordinates(moladj_parts_plate_obj.layout,part_name)]
        part_index = sample_definitions_df.index[sample_definitions_df['name'] == part_name]
        if len(part_index) > 1:
            print("WARNING: You have multiple parts with the same name!")
            break
        else:
            part_index = part_index[0]
        print(f"  1 µl \'{part_name}\'\n           from part_plate \'{part_well_location.well_name}\' -> assembly_plate \'{assembly_plate[combinations_df.loc[idx,'assembly_tube']].well_name}\'")
        smart_transfer_liquid(1,
                              part_well_location,
                              assembly_plate[combinations_df.loc[idx,'assembly_tube']],
                              asp_rate=0.2, touch_tip_before=False,touch_tip=False,
                              blow_out=True,touch_miniscus=2,
                              dispens_bot_clearance=0.8,disp_rate=0.1)
        sample_definitions_df.at[part_index,f'sample_vol_used_{str(date.today())}'] -= 1
print("\n")

Transfer of remaining, most diverse part...
  1 µl 'CDS_Eco31I(BBF10K_003292)_15nM'
           from part_plate 'B2' -> assembly_plate 'A1'
  1 µl 'CDS_EcoRI(BBF10K_003281)_15nM'
           from part_plate 'C2' -> assembly_plate 'B1'
  1 µl 'CDS_NotI(BBF10K_003300)_15nM'
           from part_plate 'D2' -> assembly_plate 'C1'
  1 µl 'CDS_PstI(BBF10K_003283)_15nM'
           from part_plate 'E2' -> assembly_plate 'D1'
  1 µl 'CDS_SapI(BBF10K_003295)_15nM'
           from part_plate 'F2' -> assembly_plate 'E1'
  1 µl 'CDS_XbaI(BBF10K_003298)_15nM'
           from part_plate 'G2' -> assembly_plate 'F1'
  1 µl 'CDS_Eco31I(BBF10K_003292)_15nM'
           from part_plate 'B2' -> assembly_plate 'G1'
  1 µl 'CDS_EcoRI(BBF10K_003281)_15nM'
           from part_plate 'C2' -> assembly_plate 'H1'
  1 µl 'CDS_NotI(BBF10K_003300)_15nM'
           from part_plate 'D2' -> assembly_plate 'A2'
  1 µl 'CDS_PstI(BBF10K_003283)_15nM'
           from part_plate 'E2' -> assembly_plate 'B2'
  1 µl 'CDS_SapI(BBF

## iii. Part dispensation (D)

In [33]:
if design_info['design type'] == 'defined':
    assemblies = [design_file_df.loc[row,:] for row in np.arange(len(design_file_df))]#[0].to_list()
    for idx, assembly_part_df in enumerate(assemblies):
        print(f"({idx+1} / {len(assemblies)}) - assemble: {combinations_df.loc[idx,'assembly_ID']}")
        for part_name in assembly_part_df.to_list():
            part_well_location = moladj_parts_plate[find_df_coordinates(moladj_parts_plate_obj.layout,part_name)]
            print(f"                      -> \'{part_name}\' from position \'{part_well_location.well_name}\'")
            smart_transfer_liquid(1,
                                  part_well_location,
                                  assembly_plate[combinations_df.loc[idx,'assembly_tube']],
                                  asp_rate=0.2, touch_tip_before=False,touch_tip=False,
                                  blow_out=True,touch_miniscus=2,
                                  dispens_bot_clearance=0.8,disp_rate=0.1)
print("\n")

In [34]:
sample_definitions_df

,name,concentration (nM),total volume (ul),moladj_parts_positions,ark_plate_format,prep_date,position_tuple,sample_vol_used_2023-06-04
0,P_T7(BBF10K_003378)_15nM,15.0,110.02,A1,96,23/05/2023,"(A, 1)",-63.68
1,P_T7_lacO(BBF10K_003379)_15nM,15.0,106.66,B1,96,23/05/2023,"(B, 1)",-63.68
2,RBS_BT1_BCD2(BBF10K_003384)_15nM,15.0,122.92,C1,96,23/05/2023,"(C, 1)",-31.84
3,RBS_BT1_BCD2_DsbA(BBF10K_003391)_15nM,15.0,169.70,D1,96,23/05/2023,"(D, 1)",-31.84
4,RBS_BT1_BCD2_OmpT(BBF10K_003392)_15nM,15.0,98.70,E1,96,23/05/2023,"(E, 1)",-31.84
5,RBS_BT1_BCD2_pelB(BBF10K_003393)_15nM,15.0,129.82,F1,96,23/05/2023,"(F, 1)",-31.84
6,Ntag1_R5(BBF10K_003399)_30nM,30.0,67.92,G1,96,23/05/2023,"(G, 1)",0.00
7,Ntag2_TEVcut(BBF10K_003418)_30nM,30.0,67.62,H1,96,23/05/2023,"(H, 1)",0.00
8,Ntag_His_TEVcut(BBF10K_003407_15nM,15.0,120.96,A2,96,23/05/2023,"(A, 2)",-63.68
9,CDS_Eco31I(BBF10K_003292)_15nM,15.0,131.38,B2,96,23/05/2023,"(B, 2)",-16.00


# Part Plate Update

In [35]:
update_moladj_parts_plate_answer = input(f"\nDo you want to update the part_plate volume information with todays part_usage?\nThis will override the saved part_plate spreadsheet.\n\n ----> yes / no ? \n\n")
if update_moladj_parts_plate_answer == 'yes':
    sample_definitions_df[f'sample_vol_before_{str(date.today())}'] = sample_definitions_df['total volume (ul)']
    sample_definitions_df['total volume (ul)'] = sample_definitions_df['total volume (ul)']+sample_definitions_df[f'sample_vol_used_{str(date.today())}']
    sample_definitions_df.drop(f'sample_vol_used_{str(date.today())}',axis=1,inplace=True)
    sample_definitions_df.to_csv(sample_definitionslayout_dir,index=False)
    print(f"\nExcellent! Now you can continue running the rest of the protocol.")
else:
    print(f"\nEnsure that this question you keep an accurate record of when you remove liquid from your molarity-adjusted parts plate.\n")


Do you want to update the part_plate volume information with todays part_usage?
This will override the saved part_plate spreadsheet.

 ----> yes / no ? 

 yes



Excellent! Now you can continue running the rest of the protocol.


# Shutdown

In [36]:
# allDone()
try:
    p20_left.drop_tip()
except:
    pass
try:
    p300_right.drop_tip()
except:
    pass
temp_mod.deactivate()
protocol.home()

In [37]:
# complete tip usage check:
_=[display(show_rack_usage(labware)[1]) for labware in labware_list if 'Opentrons 96 Tip Rack' in str(labware)]

,1,2,3,4,5,6,7,8,9,10,11,12
A,x,x,x,x,x,x,x,x,x,x,x,x
B,x,x,x,x,x,x,x,x,x,x,x,x
C,x,x,x,x,x,x,x,x,x,x,x,x
D,x,x,x,x,x,x,x,x,x,x,x,x
E,x,x,x,x,x,x,x,x,x,x,x,x
F,x,x,x,x,x,x,x,x,x,x,x,x
G,x,x,x,x,x,x,x,x,x,x,x,x
H,x,x,x,x,x,x,x,x,x,x,x,x


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,x,x,x,x,x,x,x,x,x,x,x
B,x,x,x,x,x,x,x,x,x,x,x,x
C,x,x,x,x,x,x,x,x,x,x,x,x
D,x,x,x,x,x,x,x,x,x,x,x,x
E,x,x,x,x,x,x,x,x,x,x,x,x
F,x,x,x,x,x,x,x,x,x,x,x,x
G,x,x,x,x,x,x,x,x,x,x,x,x
H,x,x,x,x,x,x,x,x,x,x,x,x


Opentrons 96 Tip Rack 20 µL on 7:
 -> 2 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,x,0,0,0,0,0,0,0,0,0,0,0
B,x,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0


Opentrons 96 Tip Rack 300 µL on 10:
 -> 0 tips used:


,1,2,3,4,5,6,7,8,9,10,11,12
A,0,0,0,0,0,0,0,0,0,0,0,0
B,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0
D,0,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0,0
F,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0,0
